In [3]:
import pandas as pd
import sys
sys.path.insert(1, '../../../scripts/')
from s3_support import *

In [43]:
exp_files = list_files("qgiv-stats-data", ".exp")

badges.exp.csv (1MB)
emailcampaign.exp.csv (44MB)
embed.exp.csv (1MB)
facebook_fundraisers.exp.csv (0MB)
form.exp.csv (2MB)
givi.exp.csv (0MB)
lists.exp.csv (0MB)
org.exp.csv (1MB)
reminders.exp.csv (0MB)
smscampaign.exp.csv (0MB)
smspledge.exp.csv (0MB)
social_settings.exp.csv (2MB)
thermometers.exp.csv (0MB)
--------------------------------------------------
Matched files: 13 files (0.0GB)
Bucket qgiv-stats-data contains 24 files (0.8GB)


In [6]:
ENTITY_TYPES = ['EVENT', 'ACCOUNT', 'TEAM', 'ORGANIZATION', 'FORM', 'SYSTEM', 'EMAILLIST', 
         'EMAILTEMPLATE', 'PAGE', 'DONATION', 'REGISTRATION', 'CONTACT', 'GLOBALACCOUNT', 
         'KIOSK', 'QGIV_EVENT', 'RESELLER', 'SMS', 'CATEGORY','PACKAGE','PRODUCT','TEAMMEMBER',
         'TEAMCAPTAIN', 'TEAMOWNER', 'ALL', 'CLASSIFICATION', 'TRANSACTION', 'PURCHASE',
         'RECIPIENT', 'SERVICE', 'RESTRICTION', 'PLEDGE', 'DEDICATION', 'PRODUCT_CATEGORY',
         'PERCENTAGE', 'FREQUENCY', 'P2P_SMS', 'PROMO_CODE', 'RECURRING', 'THERMOMETER',
         'CAMPAIGN', 'SMSPLEDGE', 'SMSLIST', 'SMSCAMPAIGN', 'BIDDER', 'AUCTION_TICKET',
         'AUCTION_ITEM_CATEGORY', 'AUCTION', 'TOKEN', 'PRICING_TEMPLATE', 'FEE', 'INVOICE',
         'REMINDER']

In [11]:
forms = get_dataframe_from_file("qgiv-stats-data", "form.exp.csv")
orgs = get_dataframe_from_file("qgiv-stats-data", "org.exp.csv")

# badges

In [46]:
BADGE_TYPES = ['ACHIEVEMENT', 'ENGAGEMENT', 'SENDING', 'FUNDRAISING_ACHIEVED',
               'TOP_FUNDRAISER', 'PROFILE_COMPLETED', 'SOCIAL_BROADCASTING', 
               'MOST_DONORS', 'TEAM_FUNDRAISING_LEADER', 'MANUAL', 
               'TEAM_FUNDRAISING_ACHIEVED', 'TEAM_TOP_FUNDRAISER', 'TEAM_MOST_DONORS']

In [47]:
badges = get_dataframe_from_file("qgiv-stats-data", "badges.exp.csv")

In [48]:
# enum names
# for some stupid ass reason badge types are indexed from 1, so -1 from enum
badges['entityType_name'] = badges['entityType'].apply(lambda x: ENTITY_TYPES[x])
badges['badgeType_name'] = badges['badgeType'].apply(lambda x: BADGE_TYPES[x-1])

# org
badges['org'] = badges['entity'].apply(lambda x: forms[forms['form']==x]['org'].iloc[0])

badges.head(3)

,entity,entityType,status,badgeType,targetEntityType,entityType_name,badgeType_name,org
0,480324,4,1,4,23,FORM,FUNDRAISING_ACHIEVED,1408
1,480324,4,1,5,23,FORM,TOP_FUNDRAISER,1408
2,480324,4,1,6,23,FORM,PROFILE_COMPLETED,1408


In [49]:
print(len(badges))
badges['entityType_name'].value_counts()

52665


FORM    52665
Name: entityType_name, dtype: int64

In [82]:
avg_form = badges.groupby('entity')['entity'].count().mean()
avg_org = badges.groupby('org')['entity'].count().mean()

"{:.2f} per org; {:.2f} per form".format(avg_org, avg_form)

'38.05 per org; 8.66 per form'

In [92]:
badges_agg = badges.groupby(['org', 'entity']).agg('count').reset_index()
badges_agg = badges_agg[['org', 'entity', 'entityType']]
badges_agg.columns = ['org', 'form', 'count']
badges_agg.head()

,org,form,count
0,9,422984,8
1,9,510018,8
2,9,511042,9
3,9,936149,9
4,9,936205,8


In [108]:
save_dataframe_to_file("qgiv-stats-data", "badges.agg.csv", badges_agg)

uploading to S3
Done


# email campaigns

In [4]:
emailcampaigns = get_dataframe_from_file("qgiv-stats-data", "emailcampaign.exp.csv")

In [7]:
emailcampaigns['entityType_name'] = emailcampaigns['entityType'].apply(lambda x: ENTITY_TYPES[x])

emailcampaigns.head(3)

,entity,entityType,template,mailingList,predicate,sent,status,created,recipients,entityType_name
0,2049,10,504256.0,"[""2049""]",271414,2015-01-21 09:30:46,1,2015-01-21 09:30:20,"[""sandra.munoz@qgiv.com"",""steve.wu@qgiv.com"",""...",REGISTRATION
1,2049,10,0.0,"[""5121""]",280634,2015-01-27 16:51:10,1,2015-01-27 16:51:00,"[""wu.steve@qgiv.com""]",REGISTRATION
2,2,10,479612.0,"[""donors""]",284725,NaN,4,2015-01-29 10:31:51,NaN,REGISTRATION


In [8]:
print(len(emailcampaigns))
emailcampaigns['entityType_name'].value_counts()

63084


REGISTRATION    60338
FORM             2746
Name: entityType_name, dtype: int64

In [12]:
emailcampaigns_agg = emailcampaigns[emailcampaigns['entityType_name']=='FORM'].groupby('entity').agg('count').reset_index()
emailcampaigns_agg = emailcampaigns_agg[['entity', 'entityType']]
emailcampaigns_agg.columns = ['form', 'count']
emailcampaigns_agg['org'] = emailcampaigns_agg['form'].apply(lambda x: forms[forms['form']==x]['org'].iloc[0])
emailcampaigns_agg.head()

,form,count,org
0,481346,3,210947
1,498757,3,210947
2,499780,3,753
3,499781,1,1663
4,499782,4,753


In [14]:
avg_form = emailcampaigns_agg['count'].mean()
avg_org = emailcampaigns_agg.groupby('org')['count'].sum().mean()

"{:.2f} per org; {:.2f} per form".format(avg_org, avg_form)

'9.15 per org; 4.17 per form'

In [15]:
save_dataframe_to_file("qgiv-stats-data", "emailcampaigns.agg.csv", emailcampaigns_agg)

uploading to S3
Done


# embeds

In [55]:
embeds = get_dataframe_from_file("qgiv-stats-data", "embed.exp.csv")

In [56]:
embeds['entityType_name'] = embeds['entityType'].apply(lambda x: ENTITY_TYPES[x])

embeds.head(3)

,id,entity,entityType,widget,widgetType,url,entityType_name
0,1,753,3,382020,1,https://secure.qgiv.com,ORGANIZATION
1,1025,753,3,383048,1,www.qgiv.com,ORGANIZATION
2,3073,9,3,625730,1,https://secure.qgiv.com/event/walkforthedogs/p...,ORGANIZATION


In [57]:
print(len(embeds))

embeds['entityType_name'].value_counts()

8691


ORGANIZATION    8691
Name: entityType_name, dtype: int64

In [58]:
avg_org = embeds.groupby('entity')['widget'].count().mean()
"{:.2f} per org".format(avg_org)

'3.80 per org'

In [98]:
embeds_agg = embeds.groupby('entity')['widget'].count().reset_index()
embeds_agg.columns = ['org', 'count']
embeds_agg.head()

,org,count
0,6,1
1,9,3
2,13,2
3,31,9
4,39,4


In [110]:
save_dataframe_to_file("qgiv-stats-data", "embeds.agg.csv", embeds_agg)

uploading to S3
Done


# facebook fundraisers

In [59]:
facebook = get_dataframe_from_file("qgiv-stats-data", "facebook_fundraisers.exp.csv")

In [60]:
facebook['entityType_name'] = facebook['entityType'].apply(lambda x: ENTITY_TYPES[x])

facebook.head(3)

,id,entity,entityType,fundraiserId,fundraiserStatus,entityType_name
0,1,831457,10,2421920984745073,0,REGISTRATION
1,2,831457,10,714875322267802,0,REGISTRATION
2,3,826595,10,917416648603188,1,REGISTRATION


In [61]:
print(len(facebook))
facebook['entityType_name'].value_counts()

592


REGISTRATION    592
Name: entityType_name, dtype: int64

# givi

In [62]:
givi = get_dataframe_from_file("qgiv-stats-data", "givi.exp.csv")
givi['org'] = givi['form'].apply(lambda x: forms[forms['form']==x]['org'].iloc[0])

In [63]:
print(len(givi))
givi.head(3)

3549


,form,website,facebookURL,twitterURL,instagramURL,org
0,932230,https://www.getgivi.com,NaN,NaN,NaN,442501
1,932254,http://viste.org/,NaN,NaN,NaN,13
2,932385,www.qgiv.com,NaN,NaN,NaN,438466


In [64]:
avg_org = givi.groupby('org')['form'].count().mean()
"{:.2f} per org".format(avg_org)

'1.00 per org'

# lists

In [65]:
lists = get_dataframe_from_file("qgiv-stats-data", "lists.exp.csv")

In [66]:
lists.head(3)

,id,created,creatingEntity,creatingEntityType,listtype,status
0,1,2019-03-18 13:50:14,442662,3,1,1
1,2,2019-03-18 13:53:23,442662,3,1,1
2,3,2019-03-18 13:54:59,443381,3,1,1


In [67]:
lists['entityType_name'] = lists['creatingEntityType'].apply(lambda x: ENTITY_TYPES[x])

print(len(lists))
lists['entityType_name'].value_counts()

541


ORGANIZATION    541
Name: entityType_name, dtype: int64

In [68]:
avg_org = lists.groupby('creatingEntity')['id'].count().mean()
"{:.2f} per org".format(avg_org)

'3.81 per org'

In [100]:
lists_agg = lists.groupby('creatingEntity')['id'].count().reset_index()
lists_agg.columns = ['org', 'count']
lists_agg.head()

,org,count
0,9,1
1,243,1
2,295,1
3,314,1
4,427,1


In [111]:
save_dataframe_to_file("qgiv-stats-data", "lists.agg.csv", lists_agg)

uploading to S3
Done


# reminders

In [69]:
reminders = get_dataframe_from_file("qgiv-stats-data", "reminders.exp.csv")

In [70]:
reminders['entityType_name'] = reminders['entityType'].apply(lambda x: ENTITY_TYPES[x])

reminders.head(3)

,id,entity,entityType,delayVal,delayIncrement,status,weight,specificTime,isRelative,reminderMsg,entityType_name
0,1,952414,4,1,days,1,1,0000-00-00 00:00:00,1,NaN,FORM
1,2,952414,4,1,days,1,2,0000-00-00 00:00:00,1,NaN,FORM
2,3,952414,4,1,days,1,3,0000-00-00 00:00:00,1,NaN,FORM


In [71]:
print(len(reminders))
reminders['entityType_name'].value_counts()

10


FORM    10
Name: entityType_name, dtype: int64

# sms campaigns

In [72]:
smscampaigns = get_dataframe_from_file("qgiv-stats-data", "smscampaign.exp.csv")

In [73]:
smscampaigns['entityType_name'] = smscampaigns['entityType'].apply(lambda x: ENTITY_TYPES[x])

smscampaigns.head(3)

,entity,entityType,created,senderType,recipients,predicate,sent,status,isScheduled,entityType_name
0,442662,3,2019-03-18 13:57:47,3,"{""QGListRows"":[2]}",686591,2019-03-18 13:58:03,2,0,ORGANIZATION
1,442662,3,2019-03-18 13:58:58,3,"{""QGListRows"":[2]}",686592,0000-00-00 00:00:00,1,1,ORGANIZATION
2,443159,3,2019-03-18 15:30:10,3,"{""QGListRows"":[3]}",686630,2019-03-18 15:30:33,2,0,ORGANIZATION


In [74]:
print(len(smscampaigns))
smscampaigns['entityType_name'].value_counts()

798


ORGANIZATION    798
Name: entityType_name, dtype: int64

In [84]:
avg_org = smscampaigns.groupby('entity')['entity'].count().mean()
"{:.2f} per org".format(avg_org)

'8.58 per org'

In [103]:
smscampaigns_agg = smscampaigns.groupby('entity').agg('count').reset_index()
smscampaigns_agg = smscampaigns_agg[['entity', 'entityType']]
smscampaigns_agg.columns = ['org', 'count']
smscampaigns_agg.head()

,org,count
0,9,12
1,295,1
2,314,1
3,427,1
4,459,1


In [112]:
save_dataframe_to_file("qgiv-stats-data", "smscampaigns.agg.csv", smscampaigns_agg)

uploading to S3
Done


# sms pledges

In [76]:
smspledges = get_dataframe_from_file("qgiv-stats-data", "smspledge.exp.csv")

In [77]:
smspledges.head(3)

,id,org,form,shortcode,startDate,endDate,amountRequired,status,created,listID
0,1,443381,940219,268945,0000-00-00 00:00:00,0000-00-00 00:00:00,0,1,2019-03-18 13:53:45,0
1,2,442662,935413,268947,2019-03-18 13:49:00,2020-03-31 13:49:00,1,1,2019-03-18 14:01:41,0
2,3,443259,939440,0,2019-03-18 14:14:00,2019-03-31 14:14:00,0,0,2019-03-18 14:15:12,0


In [86]:
avg_org = smspledges.groupby('org')['id'].count().mean()
avg_form = smspledges.groupby('form')['id'].count().mean()

"{} orgs; {} forms; {:.2f} per org; {:.2f} per form".format(len(smspledges['org'].unique()), len(smspledges['form'].unique()), avg_org, avg_form)

'390 orgs; 748 forms; 3.98 per org; 2.08 per form'

In [105]:
smspledges_agg = smspledges.groupby(['org', 'form'])['id'].count().reset_index()
smspledges_agg.columns = ['org', 'form', 'count']
smspledges_agg.head()

,org,form,count
0,9,123985,2
1,9,521282,1
2,9,625730,1
3,9,948714,1
4,62,36,1


In [113]:
save_dataframe_to_file("qgiv-stats-data", "smspledges.agg.csv", smspledges_agg)

uploading to S3
Done


# thermometers

In [79]:
thermometers = get_dataframe_from_file("qgiv-stats-data", "thermometers.exp.csv")

In [80]:
thermometers['org'] = thermometers['form'].apply(lambda x: forms[forms['form']==x]['org'].iloc[0])
thermometers.head(3)

,id,form,status,name,fundraisingGoal,beginDate,endDate,targetEntity,targetEntityType,totalRaised,primaryGoal,org
0,1,793672,1,100k,100000.0,2017-03-27 12:00:00,2018-03-27 00:00:00,0,0.0,58830.17,0,427065
1,2,793672,1,50,50000.0,2017-03-01 14:15:00,2017-03-30 14:15:00,0,NaN,10950.87,0,427065
2,3,792651,1,100,100000.0,0000-00-00 00:00:00,0000-00-00 00:00:00,0,NaN,945.00,0,427065


In [81]:
org_avg = thermometers.groupby('org')['id'].count().mean()
form_avg = thermometers.groupby('form')['id'].count().mean()
print("{} rows; {:.2f} per org; {:.2f} per form".format(len(thermometers), org_avg, form_avg))
thermometers['targetEntityType'].value_counts()

2118 rows; 2.40 per org; 1.20 per form


0.0    999
Name: targetEntityType, dtype: int64

In [107]:
thermometers_agg = thermometers.groupby(['org', 'form'])['id'].count().reset_index()
thermometers_agg.columns = ['org', 'form', 'count']
thermometers_agg.head()

,org,form,count
0,6,1,1
1,6,774210,1
2,6,945854,1
3,9,2,1
4,9,625730,1


In [114]:
save_dataframe_to_file("qgiv-stats-data", "thermometers.agg.csv", thermometers_agg)

uploading to S3
Done
